In [1]:
%load_ext autoreload
%autoreload 2
print("Starting up")
import glob
import numpy as np
import os
from copy import deepcopy as copy
from scipy.misc import imread
import sys
import ipdb
from merge_common import *
sys.path.append("/home/hthieu/UTUBEVOS2019/PReMVOS/code/")

Starting up


In [5]:
input_images = "../data/UTUBE/JPEGImages/"
first_frame_anns = "../data/UTUBE/Annotations/"
# first_frame_anns = "/home/hthieu/UTUBEVOS2019/PReMVOS/data/Annotations_debug/"
input_proposals = "../output/intermediate/ReID_proposals/"
input_optical_flow = "../output/intermediate/flow/"
output_images = "../output/final/"

curr_run_num = 0
total_to_run = 1

In [2]:
from MergeTrack.merge_functions import read_ann,read_props,calculate_scores,calculate_selected_props,\
  remove_mask_overlap,warp_proposals,update_templates,save_pngs,viz_scores,eval_video,calculate_gt_scores,\
  probabilitic_combination

from MergeTrack.refinement_net_functions import refinement_net_init, do_refinement
refinement_net = refinement_net_init()

from MergeTrack.ReID_net_functions import ReID_net_init, add_ReID
ReID_net = ReID_net_init()

creating testnet...
using bbox guidance
INFO:tensorflow:Summary name valdata/ground truth segmentation labels is illegal; using valdata/ground_truth_segmentation_labels instead.
INFO:tensorflow:Summary name valdata/bbox guidance is illegal; using valdata/bbox_guidance instead.
network:
testnet output : Using SEGMENTATION_LABELS_ORIGINAL_SIZE for calculating IoU
INFO:tensorflow:Summary name predicted labels is illegal; using predicted_labels instead.
loading model from ../weights/PReMVOS_weights/refinement_net/specific_weights/refinement_specific_weights
INFO:tensorflow:Restoring parameters from ../weights/PReMVOS_weights/refinement_net/specific_weights/refinement_specific_weights
../output/logs/ReID_net ReID_general_weights .log
number of parameters: 124,824,040
loading model from ../weights/PReMVOS_weights/ReID_net/general_weights/ReID_general_weights
INFO:tensorflow:Restoring parameters from ../weights/PReMVOS_weights/ReID_net/general_weights/ReID_general_weights


In [29]:
def do_video(video_fn, vis_frames = []):
    print("Starting", video_fn)
    final_solution = []
    templates = []
    next_props = []
    image_fn_list = sorted(glob.glob(video_fn + "*"))
    
    for image_id, image_fn in enumerate(image_fn_list):
        vis = any([x in image_fn for x in vis_frames])
        ann_fn = image_fn.replace(input_images, first_frame_anns).replace('.jpg', '.png')
        if glob.glob(ann_fn):
            # ban chat: doc file mask, duoi dang 1 file json gom toa do bbox, segmentation, embeding vector  cua re ID
            new_templates = read_ann(ann_fn)
            new_templates = add_ReID(new_templates, image_fn, ReID_net)
            templates = templates + copy(new_templates) # luu vao templates
            next_props = next_props + copy(new_templates) # luu vao next_props

        if templates:
            # doc file .json, lay danh sach cac proposal (khoang tu 8 - 12 proposal/file)
            prop_fn = image_fn.replace(input_images, input_proposals).replace('.jpg', '.json')
            proposals = next_props + read_props(prop_fn)
            
            
            print(prop_fn)
            print(len(proposals), len(templates))
            ipdb.set_trace()
            if vis:
                print("PROPOSALS CANDIDATE")
                vis_json(proposals, json_name2jpg_name(prop_fn))
            # tinh 5 scores, nhan voi trong so tuong ung
            all_scores = calculate_scores(proposals, templates)
            weighted_scores = np.dot(normalised_weights, all_scores.transpose((1, 0, 2)), )
            object_scores = np.dot(np.array([1, 1]), all_scores[:2, :, :].transpose((1, 0, 2)), )
            selected_props = calculate_selected_props(proposals, weighted_scores, templates, score_thesh, object_scores)
            
            if vis:
                print("SELECTED PROPOSALS")
                vis_json(selected_props, json_name2jpg_name(prop_fn))
            
            selected_props = remove_mask_overlap(selected_props)
            
            if vis:
                print("REMOVE OVERLAP")
                vis_json(selected_props, json_name2jpg_name(prop_fn))

            optflow_fn = image_fn.replace(input_images, input_optical_flow).replace('.jpg', '.flo')
            
            if glob.glob(optflow_fn):
                next_image_fn = image_fn_list[image_id + 1]
                next_props = warp_proposals(selected_props, optflow_fn)

                next_props = do_refinement(next_props, next_image_fn, refinement_net)
                next_props = add_ReID(next_props, next_image_fn, ReID_net)
                templates = update_templates(templates, next_props)
            else:
                print("END")
            final_solution.append((image_fn, selected_props))
            output_image_fn = image_fn.replace(input_images, output_images).replace('.jpg', '.png')
            print(output_image_fn)
            save_pngs(selected_props, output_image_fn)
        else:
            output_image_fn = image_fn.replace(input_images, output_images).replace('.jpg', '.png')
            print(output_image_fn)
            img = imread(image_fn)
            temp_props = [dict()]
            temp_props[0]['mask'] = np.zeros_like(img[:, :, 1]).astype(np.uint8)
            save_pngs(temp_props, output_image_fn, empty=True)
            

In [ ]:
score_thesh = 1e-10

weights = np.array([0.25920137, 0.22541801, 0.0775609,  0.12509281, 0.3127269 ])

normalised_weights = weights / np.sum(weights)

final_scores = []
# video_list = sorted(glob.glob(input_proposals+"0c7ba00455/")) # 2 may bay
# video_list = sorted(glob.glob(input_proposals+"fef14bb2f2/")) # 2 nguoi
# video_list = sorted(glob.glob(input_proposals+"fd237cf4fb/")) # 1 con hai cau
video_list = sorted(glob.glob(input_proposals+"f9f9aa431c/"))
vis_frames = ['00045', '00050']

#fef14bb2f
video_list = [v.replace(input_proposals,input_images) for v in video_list]
num_per = int(np.ceil(len(video_list)/total_to_run))
print(curr_run_num,num_per,curr_run_num*num_per,(curr_run_num+1)*num_per)
for video_fn in video_list[curr_run_num*num_per : (curr_run_num+1)*num_per]:
  do_video(video_fn, vis_frames)

In [47]:
from PIL import Image, ImageDraw
a = read_props('../output/intermediate/ReID_proposals/f9f9aa431c/00000.json')
img = Image.open('../data/UTUBE/JPEGImages/f9f9aa431c/00000.jpg')
drw = ImageDraw.Draw(img)
x,y,w,h = a[1]['bbox']
drw.rectangle([x,y,x+w,y+h], outline = 'black')

def cal_center(proposals):
    res = []
    for prop in proposals:
        x_c, y_c = prop['bbox'][0] + prop['bbox'][2] * 0.5, prop['bbox'][1] + prop['bbox'][3] 
        res.append((x_c, y_c))
    return res
res = cal_center(a) 
drw.point(res[1], fill = 'red')

In [51]:
print(res[1])
x,y,w,h

(408.40000915527344, 571.0000076293945)


(327.6000061035156, 480.6000061035156, 161.60000610351562, 90.4000015258789)